In [1]:
import numpy as np
import csv
from copy import copy
import re

In [2]:
with open('Export_-_2018-01-17/Oligos_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    sjoligos = list(reader)
    
with open('Export_-_2018-01-17/My_Oligos_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    kltkoligos = list(reader)
    
with open('Export_-_2018-01-17/Elim_Primers_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    elimoligos = list(reader)
    
with open('Export_-_2018-01-17/Sequetech_Primers_-_2018-01-17.csv', 'rb') as f:
    reader = csv.reader(f)
    seqoligos = list(reader)


In [3]:
alloligolist = sjoligos + kltkoligos + elimoligos + seqoligos
names = [item[0] for item in alloligolist]
sequences = [item[1] for item in alloligolist]
oligolist = zip(names, sequences)

In [242]:
import sys

In [251]:
#sys.setrecursionlimit(5000)

In [252]:
def complementary(seq):
    test_seq = seq.replace('G','c')
    test_seq = test_seq.replace('C','G')
    test_seq = test_seq.replace('c','C')
    test_seq = test_seq.replace('A','t')
    test_seq = test_seq.replace('T','A')
    test_seq = test_seq.replace('t','T')
    return test_seq

class Plasmid:
    def __init__(self, seq):
        self.sequence = seq
        self.comp = complementary(seq)
        self.blank_range = [0] * len(seq)
        self.oligo_list = {}
        self.complete_reads = []
        self.oligo_locs = {}
        self.num_iters = 0
        
    def range_int(self, st, en):
        self.target_seq = copy(self.sequence[st-1:en-1])
        self.target_range = [st-1, en] 
        
    def copy_pattern(self, oligos):
        #find each primers range
        #odd= start, even = end
        count = 1
        for i, o in enumerate(oligos):
            if self.blank_range[o[4][0]] == 0:
                self.blank_range[o[4][0]] = count
                self.oligo_list[count] = [o[0], o[3]]
                self.oligo_locs[count] = [o[4][0], o[4][1]]
                
                count += 1
                self.blank_range[o[4][1]] = count
                count += 1
            else:
                new_odd, new_even = self.adjuster(o[4][0], o[4][1])
                
                self.blank_range[new_odd] = count
                self.oligo_list[count] = [o[0], o[3]]
                self.oligo_locs[count] = [new_odd, new_even]
                
                count += 1
                self.blank_range[new_even] = count
                count += 1

    def adjuster(self, st, en):
        #if one of the original locations is occupied by another primer's range
        mag = 1
        vec = 1
        
        while True:
            new_st = st + (mag*vec)
            new_en = en + (mag*vec)
            
            if self.blank_range[new_st] == 0 and self.blank_range[new_en] == 0:
                print mag*vec
                return new_st, new_en
            elif vec > 0:
                vec *= -1
            else:
                vec *= -1
                mag += 1
    
    def find_last_odd(self, curr, lowlim):
        all_rem_odds = [x for x in self.blank_range[lowlim+1:curr] if x%2==1]
        
        if all_rem_odds == []:
            return ['','']
        
        last_odd = all_rem_odds[-1]
        new_curr = plasmid.blank_range.index(last_odd)
        return [last_odd, new_curr]
            
        
    
    def find_full_seq(self, curr_ind, low_ind, path):
        
        [next_odd, next_low] = self.find_last_odd(curr_ind, low_ind)
        
        self.num_iters += 1
        #print self.num_iters
    
        #print [next_odd, next_low]
        
        if next_odd != '' and (next_low > self.target_range[1] or next_low < self.target_range[0] - 600):
            #print 'exit'
            return
        
        if next_odd != '':
            path.append(next_odd)
            #print path
            curr_ind = self.blank_range.index(next_odd + 1)
            
            if curr_ind >= self.target_range[1]:
                self.complete_reads.append(list(path))
                #print 'reads', self.complete_reads
                
                #print 'save', path
                curr_ind = self.oligo_locs[path[-1]][0]
                del path[-1]
                low_ind = self.oligo_locs[path[-1]][0]
                #print 'next', path
                
                self.find_full_seq(curr_ind, low_ind, path)
            
            else:
                self.find_full_seq(curr_ind, next_low, path)
                
        else:
            curr_ind = self.oligo_locs[path[-1]][0]
            del path[-1]
            low_ind = self.oligo_locs[path[-1]][0]
            
            self.find_full_seq(curr_ind, low_ind, path)
            
        
    def print_reads(self):
        print 2
    
class Oligos:
    def __init__(self, oliglist):
        self.oligos = self.remove_size(oliglist)        
        self.oligos = zip([item[0] for item in self.oligos], [item[1].upper() for item in self.oligos], [item[1].upper()[::-1] for item in self.oligos])
        
        self.binders = []
        self.find_bind()
        
    def remove_size(self, oliglist):
        return [x for x in oliglist if len(x[1]) < 35 and len(x[1]) > 15]

    def find_bind(self):
        for i, o in enumerate(self.oligos):
            bind = [m.end() for m in re.finditer(o[1], plasmid.sequence)]
            bind_rev = [m.start() for m in re.finditer(o[2], plasmid.comp)]
        
            if len(bind) + len(bind_rev) == 1:
                direction, ran = self.pattern(bind, bind_rev)
                self.binders.append((o[0], o[1], o[2], direction, ran))
        
    def pattern(self, b, br):
        if b == []:
            return '-', [br[0]-750, br[0]-50]
        elif br == []:
            return '+', [b[0]+50, b[0]+750]
    
    def binding_locations(self, targ_sequence):
        print 1

In [253]:
with open('plasmid.txt', 'r') as f:
    plasmid_sequence = f.read()

plasmid = Plasmid(plasmid_sequence.upper())
range_start = 906
range_end = 2013
plasmid.range_int(range_start,range_end)

In [254]:
li, li2 = plasmid.find_last_odd(401,400)
plasmid.blank_range.index(163)

print li, li2

ValueError: 163 is not in list

In [255]:
o = Oligos(oligolist)
print o.binders[4][0]

oSJ7


In [256]:
plasmid.copy_pattern(o.binders)
print plasmid.blank_range
print plasmid.oligo_list

1
1
1
1
1
-1
1
1
-1
1
1
1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 138, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 139, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [257]:
[906, 1713]
print plasmid.blank_range.index(133)
print plasmid.blank_range.index(87)
print plasmid.blank_range.index(25)

1062
1013
901


In [258]:

plasmid.find_full_seq(906, 1, [])

IndexError: list index out of range

In [259]:
print plasmid.num_iters

2484


In [105]:
print len(o.binders)

for i,oli in enumerate(o.binders):
    print oli

83
('oSJ11', 'CGCATCGAGCTGAAGGGCAT', 'TACGGGAAGTCGAGCTACGC', '+', [4737, 5437])
('oSJ12', 'TTAGGCGGCGGTCACGAACT', 'TCAAGCACTGGCGGCGGATT', '-', [4218, 4918])
('oSJ1', 'GCTGGAGTTCGTGACCGCCGCCTAA', 'AATCCGCCGCCAGTGCTTGAGGTCG', '+', [5038, 5738])
('oSJ3', 'GTTAGCCTTGATGCCGTTCTTCTGCTTGTCGGCCG', 'GCCGGCTGTTCGTCTTCTTGCCGTAGTTCCGATTG', '-', [4011, 4711])
('oSJ2', 'AGTCGATGCCCTTCAGCTCGATGCG', 'GCGTAGCTCGACTTCCCGTAGCTGA', '-', [3917, 4617])
('oSJ7', 'CAGGATGTTGCCGTCCTCCTTGAAGTCGATG', 'GTAGCTGAAGTTCCTCCTGCCGTTGTAGGAC', '-', [3934, 4634])
('oSJ74', 'TTGCTCAGGGCGGACTGATGGCTCA', 'ACTCGGTAGTCAGGCGGGACTCGTT', '-', [4155, 4855])
('oSJ78', 'TTGCCGTCCTCCTTGAAGTC', 'CTGAAGTTCCTCCTGCCGTT', '-', [3938, 4638])
('oSJ29', 'TAAGCGGCCGCACTCGAGAT', 'TAGAGCTCACGCCGGCGAAT', '+', [5061, 5761])
('oSJ8', 'CAGTCCGCCCTGAGCAAAGACCCCAA', 'AACCCCAGAAACGAGTCCCGCCTGAC', '+', [4989, 5689])
('oSJ9', 'GGATCTTGAAGTTAGCCTTGATGCCGTTCTTCTGC', 'CGTCTTCTTGCCGTAGTTCCGATTGAAGTTCTAGG', '-', [4021, 4721])
('oSJ36', 'GGTGGAGCCTGCTTTTTTGT'

In [ ]:
for oligo in oligolist:
    [m.start() for m in re.finditer(, plasmid_sequence)]

In [201]:
oSJ5.sequence

'CATGGTGGAGCCTGCTTTTTTGTAC'

In [106]:
testuniv = range(0,10)
nouniv = np.arange(101,200)
subs = []
for i in testuniv:
    subs.append(range(i,i+3))
    

In [107]:
print subs

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [9, 10, 11]]


In [103]:
new = []
new = [s for s in subs if s != subs[0]]
print new

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22], [14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [15, 16, 17, 18, 19, 20, 21, 22, 23, 24], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [17, 18, 19, 20, 21, 22, 23, 24, 25, 26], [18, 19, 20, 21, 22, 23, 24, 25, 26, 27], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [23, 24, 25, 26, 27, 28, 29, 30, 31, 32], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [25, 26, 27, 28, 29, 30, 31, 32, 33,

In [152]:
def recur(univ, subsets, templist):
    remuniv = copy(univ)
    for s in subsets:
        overlap = list(set(s) & set(remuniv))
        
        if overlap != []:
            remuniv = [x for x in univ if x not in overlap]
            remsets = [l for l in subsets if l != s]
            templist.append(s)
            
            if remuniv == []:
                print "one"
                answer.append(templist)
                return templist
            else:
                new = recur(remuniv, remsets, templist)
                del templist[-1]
                

In [153]:
answer = []
tempin = []
recur(testuniv, subs, tempin)

one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one


In [155]:
print answer[0]

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [6, 7, 8], [5, 6, 7], [7, 8, 9], [8, 9, 10], [5, 6, 7], [9, 10, 11], [5, 6, 7], [6, 7, 8], [5, 6, 7], [4, 5, 6], [6, 7, 8], [7, 8, 9], [8, 9, 10], [6, 7, 8], [4, 5, 6], [7, 8, 9], [4, 5, 6], [8, 9, 10], [4, 5, 6], [9, 10, 11], [6, 7, 8], [5, 6, 7], [7, 8, 9], [4, 5, 6], [8, 9, 10], [4, 5, 6], [5, 6, 7], [5, 6, 7], [4, 5, 6], [6, 7, 8], [5, 6, 7], [9, 10, 11], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [5, 6, 7], [4, 5, 6], [6, 7, 8], [7, 8, 9], [4, 5, 6], [8, 9, 10], [4, 5, 6], [4, 5, 6], [3, 4, 5], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [6, 7, 8], [5, 6, 7], [7, 8, 9], [8, 9, 10], [5, 6, 7], [9, 10, 11], [5, 6, 7], [6, 7, 8], [5, 6, 7], [3, 4, 5], [6, 7, 8], [7, 8, 9], [8, 9, 10], [6, 7, 8], [3, 4, 5], [7, 8, 9], [3, 4, 5], [8, 9, 10], [3, 4, 5], [9, 10, 11], [6, 7, 8], [3, 4, 5], [5, 6, 7], [5, 6, 7], [3, 4, 5], [7, 8, 9], [7, 8, 9], [3, 4, 5], [8, 9, 10], [3, 4, 5], [5, 6, 7],

In [169]:
j=0
for i, o in enumerate(oligolist):
    bind = [m.start() for m in re.finditer(o[1].upper(), plasmid.sequence)]
    bind_rev = [m.start() for m in re.finditer(o[1].upper()[::-1], plasmid.comp)]

    if len(bind) + len(bind_rev) > 0:
        j +=1
        print i
        print o[0], o[1], bind, bind_rev
        print zip(o[0], o[1], bind, bind_rev)
        
print j

2
oSJ11 CGCATCGAGCTGAAGGGCAT [4667] []
[]
3
oSJ12 TTAGGCGGCGGTCACGAACT [] [4968]
[]
8
oSJ1 GCTGGAGTTCGTGACCGCCGCCTAA [4963] []
[]
9
oSJ3 GTTAGCCTTGATGCCGTTCTTCTGCTTGTCGGCCG [] [4761]
[]
10
oSJ2 AGTCGATGCCCTTCAGCTCGATGCG [] [4667]
[]
25
oSJ7 caggatgttgccgtcctccttgaagtcgatg [] [4684]
[]
31
oSJ74 TTGCTCAGGGCGGACTGATGGCTCA [] [4905]
[]
35
oSJ78 ttgccgtcctccttgaagtc [] [4688]
[]
37
oSJ29 TAAGCGGCCGCACTCGAGAT [4991] []
[]
43
oSJ8 cagtccgccctgagcaaagaccccaa [4913] []
[]
44
oSJ9 ggatcttgaagttagccttgatgccgttcttctgc [] [4771]
[]
49
oSJ36 GGTGGAGCCTGCTTTTTTGT [] [1134]
[]
65
oSJ40 CTTTGTACAAAAAAGCAGGC [1128] []
[]
70
oSJ55 GGTGGAGCCTGCTTTTTTGTACAAA [] [1129]
[]
76
oSJ62 Ccctggggagagaggtcggt [] [1651]
[]
77
oSJ77 TGAGCCATCAGTCCGCCCTG [4905] []
[]
81
oSJ82 CCAACTCTTTTTCCGAAGGTAACTGGCTTCAGCAGAGCGC [132] []
[]
82
oSJ83 GAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATC [] [120]
[]
102
oSJ104 gaattactgagttgatgggt [2475] []
[]
104
oSJ105 attcaagttggaactggtgg [3331] []
[]
107
oSJ106 AGCAAAGACCCCAACGAGAA [4925] 